# 0.Register
the xxEnv, env_cfg and agent_cfg is how to convey in this framework

- 首先要有个概念：我们的env_cfg继承了DirectRLEnvCfg，agent_cfg继承了RslRlOnPolicyRunnerCfg，xxEnv继承了DirectRLEnv，也就继承了一些方法（函数）和一些默认属性property。当然也有些方法只有声明，没有实现implement，例如the '_get_observations' method就是在我们的xxEnv中具体实现的。
- 其次，通过gym.register(id="xxtask", entry_point="xxEnv", kwargs={}, )注册了env。其中xxEnv是继承了DirectRLEnv的类，这个类中定义了env的属性和函数。

- 然后，由gym.make(id, cfg=env_cfg, )创建的env，又通过RslRlVecEnvWrapper()进行了wrap
- 最后传入了OnPolicyRunner()

scripts/rsl_rl/train.py
```python
@hydra_task_config(args_cli.task, "rsl_rl_cfg_entry_point")
def main(env_cfg: ManagerBasedRLEnvCfg | DirectRLEnvCfg | DirectMARLEnvCfg, agent_cfg: RslRlOnPolicyRunnerCfg):
    # ...
    # create isaac environment
    env = gym.make(args_cli.task, cfg=env_cfg, render_mode="rgb_array" if args_cli.video else None)
    # ...
    # wrap around environment for rsl-rl
    env = RslRlVecEnvWrapper(env)
    # create runner from rsl-rl
    runner = OnPolicyRunner(env, agent_cfg.to_dict(), log_dir=log_dir, device=agent_cfg.device)
    # ...
    # run training
    runner.learn(num_learning_iterations=agent_cfg.max_iterations, init_at_random_ep_len=True)
    # close the simulator
    env.close()
```

## 0.1 gym.register()
Gym 使用一个注册表（registry）来管理所有可用的环境。每个环境都有一个唯一的 ID，并且在注册时指定了如何实例化该环境: `entry_point`。

通常，环境通过 `gym.register` 函数进行注册。
```python
"""Registers an environment in gymnasium with an ``id`` to use with :meth:`gymnasium.make` with the ``entry_point`` being a string or callable for creating the environment.
It takes arbitrary keyword arguments, which are passed to the :class:`EnvSpec` ``kwargs`` parameter."""
```

.e.g例如
```python
import gymnasium as gym
from . import agents
from .zbot6b_env_v0 import ZbotBEnv, ZbotBEnvCfg
##
# Register Gym environments.
##
gym.register(
    id="Zbot-6b-walking-v0",
    entry_point="Zbot.tasks.moving.zbot6b_direct:ZbotBEnv",
    disable_env_checker=True,
    kwargs={
        "env_cfg_entry_point": ZbotBEnvCfg, 
        "rsl_rl_cfg_entry_point": f"{agents.__name__}.rsl_rl_ppo_cfg:ZbotSBFlatPPORunnerCfg",
    },
)
```

## 0.2 env_cfg & agent_cfg
decorator 负责解析并提供 env_cfg 和 agent_cfg 

scripts/rsl_rl/train.py
```python
@hydra_task_config(args_cli.task, "rsl_rl_cfg_entry_point")
def main(env_cfg: ManagerBasedRLEnvCfg | DirectRLEnvCfg | DirectMARLEnvCfg, agent_cfg: RslRlOnPolicyRunnerCfg):
    # ...
```

```python
def hydra_task_config(task_name: str, agent_cfg_entry_point: str) -> Callable:
    #...
    env_cfg, agent_cfg = register_task_to_hydra(task_name, agent_cfg_entry_point)
    #...
```

```python
def register_task_to_hydra(
    task_name: str, agent_cfg_entry_point: str
) -> tuple[ManagerBasedRLEnvCfg | DirectRLEnvCfg, dict]：
    # ...
    env_cfg = load_cfg_from_registry(task_name, "env_cfg_entry_point")
    agent_cfg = load_cfg_from_registry(task_name, agent_cfg_entry_point)
    # ...
```

```python
def load_cfg_from_registry(task_name: str, entry_point_key: str) -> dict | object:
    """It supports both YAML and Python configuration files.
    If the entry point is a YAML file, it is parsed into a dictionary.
    If the entry point is a Python class, it is instantiated and returned."""
    # obtain the configuration entry point
    cfg_entry_point = gym.spec(task_name).kwargs.get(entry_point_key)
    # 如果 cfg_entry_point 是一个以 .yaml 结尾的字符串
    # 如果 cfg_entry_point 是可调用的（例如是一个函数或类）
    # 如果 cfg_entry_point 是字符串（格式为 "module_name:attr_name"）
    # ...
```

## 0.3 gym.make()

`gym.make` 是 Gym 库中的一个核心函数，用于创建和初始化环境实例。它根据给定的环境 ID 和其他参数来实例化相应的环境类。
```python
import gymnasium as gym
gym.register(
    id='CustomEnv-v0',
    entry_point='custom_env_module:CustomEnv',
    kwargs={
        "env_cfg_entry_point": xxEnvCfg, 
        "rsl_rl_cfg_entry_point": "xxAgentCfg",
    }
)
```
```python
env = gym.make(args_cli.task, cfg=env_cfg, render_mode="rgb_array" if args_cli.video else None)
```
- 解析环境 ID

当调用 `gym.make` 时，Gym 会解析传入的 `args_cli.task`，并从注册表中查找对应id的环境条目。如果找到了匹配的条目，则使用其 `entry_point` 来实例化环境。
- 实例化环境

`gym.make` 会根据找到的 `entry_point` 动态导入模块并实例化环境类。例如，如果 `entry_point` 是 `'custom_env_module:CustomEnv'`，那么 Gym 会导入 `custom_env_module` 模块并调用 `CustomEnv` 构造函数来创建环境实例。
- 传递参数

除了环境 ID，`gym.make` 还可以接受额外的关键字参数，这些参数会传递给环境构造函数。例如：`cfg=env_cfg` 和 `render_mode="rgb_array" if args_cli.video else None` 是传递给 `CustomEnv` 构造函数的额外参数。
> 注意：`env_cfg`又是在# 0.2 中通过解析注册表参数得到的。
> 
> `gym.spec(task_name).kwargs.get("env_cfg_entry_point")`
- 总结

`gym.make` 通过注册表查找环境 ID，动态导入并实例化相应的环境类，并将额外的参数传递给环境构造函数。最终返回一个初始化好的环境实例，供用户使用。

gymnasium/envs/registration.py
```python
env_creator = env_spec.entry_point | env_creator = _load_env_creator(env_spec.entry_point)
# env_creator = custom_env_module.CustomEnv .e.g 
env = env_creator(**env_spec_kwargs)  # 使用解包后的 env_spec_kwargs 创建环境实例
```

## 0.4 *args & **kwargs
在 Python 中，函数或方法的参数可以分为以下几类：
- **位置参数（positional arguments）**：必须按顺序传递，不能省略（除非有默认值）。
- **关键字参数（keyword arguments）**：通过参数名传递，顺序无关紧要。
- **默认参数（default arguments）**：提供默认值，使某些参数成为可选参数。
- **可变长度参数（variadic arguments）**：`*args` 和 `**kwargs` 提供了极大的灵活性，允许函数接受任意数量的位置参数和关键字参数。

### 1. 位置参数（Positional Arguments）
- **定义**：必须按照定义时的顺序传递给函数。
- **特点**：不能省略，除非有默认值。
- **示例**：
```python
def greet(name, greeting):
    print(f"{greeting}, {name}!")

greet("Alice", "Hello")  # 输出: Hello, Alice!
```

### 2. 关键字参数（Keyword Arguments）
- **定义**：通过参数名传递给函数，顺序无关紧要。
- **特点**：可以提高代码的可读性，避免混淆参数的顺序。
- **示例**：
```python
def greet(name, greeting):
    print(f"{greeting}, {name}!")

greet(greeting="Hi", name="Bob")  # 输出: Hi, Bob!
```

### 3. 默认参数（Default Arguments）
- **定义**：为参数提供默认值，如果调用时未提供该参数，则使用默认值。
- **特点**：使某些参数成为可选参数。
- **示例**：
```python
def greet(name, greeting="Hello"):
    print(f"{greeting}, {name}!")

greet("Alice")  # 输出: Hello, Alice!
greet("Bob", "Hi")  # 输出: Hi, Bob!
```

### 4. 可变长度参数（Variadic Arguments）
- **定义**：允许函数接受任意数量的位置参数和/或关键字参数。
- **特点**：增加了函数的灵活性。
- **分类**：
  - `*args`：捕获所有未明确指定的位置参数，作为元组传递。使用场景：当你不确定函数需要多少个位置参数时，或者希望函数能够接受任意数量的位置参数。
  - `**kwargs`：捕获所有未明确指定的关键字参数，作为字典传递。使用场景：当你不确定函数需要多少个关键字参数时，或者希望函数能够接受任意数量的关键字参数。
- **示例**：
```python
def greet(*args, **kwargs):
    if args:
        for arg in args:
            print(f"Positional argument: {arg}")
    if kwargs:
        for key, value in kwargs.items():
            print(f"Keyword argument: {key} = {value}")

greet("Alice", "Bob", greeting="Hi", farewell="Goodbye")
# 输出:
# Positional argument: Alice
# Positional argument: Bob
# Keyword argument: greeting = Hi
# Keyword argument: farewell = Goodbye
```

## 0.5 解包&传参
在 Python 中，**dict 是一种用于解包字典的语法，它允许你将字典中的键值对作为关键字参数传递给函数或方法。

```python
env = gym.make(args_cli.task, cfg=env_cfg, render_mode="rgb_array" if args_cli.video else None)
```
由make的函数签名可知：
- `args_cli.task`是位置参数，匹配给第一个参数`id`
- `cfg=env_cfg` 和 `render_mode="rgb_array"` 是可变长度参数，会被捕获到字典中，即 kwargs = {"cfg": env_cfg, "render_mode": "rgb_array"}

gymnasium/envs/registration.py
```python
def make(
    id: str | EnvSpec,
    max_episode_steps: int | None = None,
    autoreset: bool | None = None,
    apply_api_compatibility: bool | None = None,
    disable_env_checker: bool | None = None,
    **kwargs: Any,
) -> Env:
    # Get the env spec
    # env_spec = id | env_spec = _find_spec(id)
    # ...
    # Update the env spec kwargs with the `make` kwargs
    env_spec_kwargs = copy.deepcopy(env_spec.kwargs)
    # 前面的 `cfg=env_cfg` 和 `render_mode="rgb_array"` 被捕获到字典中，即 kwargs = {"cfg": env_cfg, "render_mode": "rgb_array"}
    # 再更新到字典 env_spec_kwargs 中
    env_spec_kwargs.update(kwargs)
    # Load the environment creator
    # env_creator = env_spec.entry_point | env_creator = _load_env_creator(env_spec.entry_point)
    # env_creator = custom_env_module.CustomEnv .e.g 
    # ...
    env = env_creator(**env_spec_kwargs)  # 使用解包后的 env_spec_kwargs 创建环境实例
```

xxEnv构造函数签名
```python
def __init__(self, cfg: ZbotBEnvCfg, render_mode: str | None = None, **kwargs):
```
- **`cfg`**：位置参数，类型为 `ZbotBEnvCfg`，是必须传递的配置对象。
- **`render_mode`**：关键字参数，默认值为 `None`，表示渲染模式，可以省略。
- **`**kwargs`**：可变长度关键字参数，捕获所有其他未明确指定的关键字参数。


当调用构造函数，并**使用字典解包传递参数**时，Python 会根据参数名自动匹配到方法签名中的相应参数，并将剩余的参数放入 `kwargs` 字典中。

例如：
```python
env = xxEnv(
    **{
        "some_other_param1": "value1"
        "cfg": my_cfg,
        "render_mode": "rgb_array",
        "some_other_param2": value2
    }
)
```
- `cfg=my_cfg` 匹配到 `cfg` 参数。
- `render_mode="rgb_array"` 匹配到 `render_mode` 参数。
- `some_other_param1="value1"` 、`some_other_param2=value2`捕获到 `kwargs` 字典中。

# 1.log_dir

scripts/rsl_rl/train.py
```python
def main(env_cfg: ManagerBasedRLEnvCfg | DirectRLEnvCfg | DirectMARLEnvCfg, agent_cfg: RslRlOnPolicyRunnerCfg):
    # ...
    # specify directory for logging experiments
    log_root_path = os.path.join("logs", "rsl_rl", agent_cfg.experiment_name)
    log_root_path = os.path.abspath(log_root_path)
    print(f"[INFO] Logging experiment in directory: {log_root_path}")
    # specify directory for logging runs: {time-stamp}_{run_name}
    log_dir = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    if agent_cfg.run_name:
        log_dir += f"_{agent_cfg.run_name}"
    log_dir = os.path.join(log_root_path, log_dir)
    # ...
```

# 2.OnPolicyRunner
runner = OnPolicyRunner(env, agent_cfg.to_dict(), log_dir=log_dir, device=agent_cfg.device)

## 2.1

```python
class RslRlOnPolicyRunnerCfg:
    """Configuration of the runner for on-policy algorithms."""

    seed: int = 42
    """The seed for the experiment. Default is 42."""

    device: str = "cuda:0"
    """The device for the rl-agent. Default is cuda:0."""
```

所以 agent_cfg.device = "cuda:0"

## 2.2
line 29、93 `obs, extras = self.env.get_observations()`其中get_observations()这个attribute

是由于train.py中`env = RslRlVecEnvWrapper(env)`进行了wrap

```python
# ~/IsaacLab/source/extensions/omni.isaac.lab_tasks/omni/isaac/lab_tasks/utils/wrappers/rsl_rl/vecenv_wrapper.py
class RslRlVecEnvWrapper(VecEnv):
    # ...
    def get_observations(self) -> tuple[torch.Tensor, dict]:
        """Returns the current observations of the environment."""
        if hasattr(self.unwrapped, "observation_manager"):
            obs_dict = self.unwrapped.observation_manager.compute()
        else:
            obs_dict = self.unwrapped._get_observations()
        return obs_dict["policy"], {"observations": obs_dict}
    # ...
```

.e.g例如
```python
class ZbotBEnv(DirectRLEnv):
    cfg: ZbotBEnvCfg
    def __init__(self, cfg: ZbotBEnvCfg, render_mode: str | None = None, **kwargs):
        super().__init__(cfg, render_mode, **kwargs)
        # ...
    def _get_observations(self) -> dict:
        obs = torch.cat(
            (
                self.body_quat[:,0].reshape(self.scene.cfg.num_envs, -1),
                self.body_quat[:,3].reshape(self.scene.cfg.num_envs, -1),
                self.body_quat[:,6].reshape(self.scene.cfg.num_envs, -1),
                self._commands,
                self.joint_vel,
                self.joint_pos,
                # 4*(3)+3+6+6
            ),
            dim=-1,
        )
        observations = {"policy": obs}
        return observations
```

## 2.3 init_at_random_ep_len
line 89 `init_at_random_ep_len` 在train.py中传入:

`runner.learn(num_learning_iterations=agent_cfg.max_iterations, init_at_random_ep_len=True)`
```python
    episode_length_buf: torch.Tensor  # current episode duration
    """Buffer for current episode lengths."""
```

```python
class DirectRLEnv(gym.Env):
    # ...
    def __init__(self, cfg: DirectRLEnvCfg, render_mode: str | None = None, **kwargs):
        # ...
        self.episode_length_buf = torch.zeros(self.num_envs, device=self.device, dtype=torch.long)
        # ...
```

```python
class ZbotBEnv(DirectRLEnv):
    # ...
    def _reset_idx(self, env_ids: torch.Tensor | None):
        if env_ids is None or len(env_ids) == self.num_envs:
            env_ids = self.zbots._ALL_INDICES
        self.zbots.reset(env_ids)
        super()._reset_idx(env_ids)
        if len(env_ids) == self.num_envs:
            # Spread out the resets to avoid spikes in training when many environments reset at a similar time
            # 分散重置以避免在许多环境同时重置时出现训练峰值
            self.episode_length_buf[:] = torch.randint_like(
                self.episode_length_buf, high=int(self.max_episode_length)
            )
        # ...
```

```python
class OnPolicyRunner:
    # ...
    def learn(self, num_learning_iterations: int, init_at_random_ep_len: bool = False):
        # ...
        if init_at_random_ep_len:
            self.env.episode_length_buf = torch.randint_like(
                self.env.episode_length_buf, high=int(self.env.max_episode_length)
            )
        # ...
```

## 2.4
line109 `with torch.inference_mode()` 

在 rollout 阶段，模型生成动作并与环境交互，目的是收集数据以供后续训练使用。此时禁用梯度计算，可以优化性能并确保数据收集过程的可靠性。

其中：
- `self.alg.act(obs, critic_obs)` 生成动作并与环境交互。
- `self.env.step(actions)` 执行动作并返回新的观测、奖励、完成标志和其他信息。
- `self.obs_normalizer(obs)` 和 `self.critic_obs_normalizer(infos["observations"]["critic"])` 对观测进行归一化处理。
- `self.alg.process_env_step(rewards, dones, infos)` 处理环境反馈，准备后续的训练数据。

训练数据的收集真正靠的是`self.alg.process_env_step(rewards, dones, infos)`传入，

而外部的这些变量更多是用于tensorboard和terminal的展示。

## 2.5
line 112 `obs, rewards, dones, infos = self.env.step(actions)`使用四个参数接收返回值，也是因为`RslRlVecEnvWrapper()`

原本有5个返回值
```python
class DirectRLEnv(gym.Env):
    # ...
    def step(self, action: torch.Tensor) -> VecEnvStepReturn:
        """Returns:
            A tuple containing the observations, rewards, resets (terminated and truncated) and extras.
        """
        # ...
        # return observations, rewards, resets and extras
        return self.obs_buf, self.reward_buf, self.reset_terminated, self.reset_time_outs, self.extras
    # ...
```

```python
class RslRlVecEnvWrapper(VecEnv):
    #...
    def step(self, actions: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, dict]:
        # record step information
        obs_dict, rew, terminated, truncated, extras = self.env.step(actions)
        # compute dones for compatibility with RSL-RL
        dones = (terminated | truncated).to(dtype=torch.long)
        # move extra observations to the extras dict
        obs = obs_dict["policy"]
        extras["observations"] = obs_dict
        # move time out information to the extras dict
        # this is only needed for infinite horizon tasks
        if not self.unwrapped.cfg.is_finite_horizon:
            extras["time_outs"] = truncated

        # return the step information
        return obs, rew, dones, extras
    # ...
```

## 2.6
```python
rewbuffer.extend(cur_reward_sum[new_ids][:, 0].cpu().numpy().tolist())
lenbuffer.extend(cur_episode_length[new_ids][:, 0].cpu().numpy().tolist())
```
如果是比较稳定没有terminate了的话，大多数env达到max_episode_length，即time_out时，才会reset。

`max_episode_length = math.ceil(self.cfg.episode_length_s / (self.cfg.sim.dt * self.cfg.decimation))` 1000 = 20 / (1 / 200 * 4) .e.g

所以后面mean_episode_length ≈ max_episode_length。

### 2.6.1 cur_reward_sum[new_ids][:, 0]

#### nonzero(as_tuple=False)
`new_ids = (dones > 0).nonzero(as_tuple=False)`，`as_tuple=False`表示返回结果是一个二维张量，而不是一个元组。

要么就像这样：`reset_env_ids = self.reset_buf.nonzero(as_tuple=False).squeeze(-1)`（在`DirectRLEnv.step()`中）

对于一维张量，nonzero 返回的张量的形状是 (N, 1)，其中 N 是非零元素的数量。
```python
print(cur_reward_sum.shape)  # torch.Size([64])
print(new_ids.shape)  # torch.Size([2, 1])
print(new_ids)  # tensor([[1], [3]], device='cuda:0')
print(cur_reward_sum[new_ids].shape)  # torch.Size([2, 1])  # 升维了
```
通过高级索引，改变了shape，从一维变成了二维，因此`cur_reward_sum[new_ids][:, 0]`才没抛出异常。（不能对一维进行[:, 0]）

#### 高级索引（Advanced Indexing）

高级索引允许你使用整数数组来选择张量中的元素。高级索引的行为与基本切片（basic slicing）不同，它会返回一个与**索引数组形状相同的新张量**。

In [4]:
import torch
# 初始化 A 和 new_ids
A = torch.randn(64, dtype=torch.float, device='cuda')
print(A.shape)  # 输出: torch.Size([64])
new_ids = torch.tensor([[1], [3], [6], [8]], dtype=torch.long, device='cuda')
# 计算 A[new_ids]
result = A[new_ids]
print(result)
print(result.shape)  # 输出: torch.Size([4, 1])

new_ids_2 = torch.tensor([[1, 3], [6, 8], [6, 8], [6, 8]], dtype=torch.long, device='cuda')
result_2 = A[new_ids_2]
print(result_2)
print(result_2.shape)  # 输出: torch.Size([4, 2])

torch.Size([64])
tensor([[-2.1058],
        [ 0.1667],
        [ 1.7387],
        [-0.5361]], device='cuda:0')
torch.Size([4, 1])
tensor([[-2.1058,  0.1667],
        [ 1.7387, -0.5361],
        [ 1.7387, -0.5361],
        [ 1.7387, -0.5361]], device='cuda:0')
torch.Size([4, 2])


### 2.6.2 deque
虽然rewbuffer每个step都在更新(除非某一步所有env都没有reset)，但每个iteration(16steps .e.g)才计算rewbuffer平均值并更log一次

同时rewbuffer的maxlen=100，前期一个step的reset的env数量多，可能都不够装，只能记录后100个reset的env的值

也就是说log时，可能只用到了一轮iteration中最后一个step的后100个reset_env的值，来计算平均值

当然，通过前面提到的`init_at_random_ep_len`机制，reset的env分散了，不会总是固定的那100个env

In [9]:
from collections import deque
import statistics

rewbuffer = deque(maxlen=100)
rewbuffer.extend([1, 1, 1, 1])
rewbuffer.extend([1, 1, 1, 1, 1])
rewbuffer.extend([1, 1, 1])
print(rewbuffer)
print(statistics.mean(rewbuffer))

small_buffer = deque(maxlen=3)
small_buffer.extend([-1, 0, 1, 2])
print(small_buffer)
small_buffer.extend([3, 4])
print(small_buffer)
print(statistics.mean(small_buffer))

deque([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], maxlen=100)
1
deque([0, 1, 2], maxlen=3)
deque([2, 3, 4], maxlen=3)
3


## 2.7
line154 `self.log(locals())`， log的详解见#3

↓

line205 `self.writer.add_scalar("Train/mean_reward", statistics.mean(locs["rewbuffer"]), locs["it"])`

### 2.7.1 statistics.mean()
通过 `_sum` 函数计算总和与元素个数，并使用 `_convert` 函数确保结果类型与输入一致。如分数 (Fraction) 和十进制数 (Decimal)。

```python
def mean(data):
    """Return the sample arithmetic mean of data.

    >>> mean([1, 2, 3, 4, 4])
    2.8

    >>> from fractions import Fraction as F
    >>> mean([F(3, 7), F(1, 21), F(5, 3), F(1, 3)])
    Fraction(13, 21)

    >>> from decimal import Decimal as D
    >>> mean([D("0.5"), D("0.75"), D("0.625"), D("0.375")])
    Decimal('0.5625')

    If ``data`` is empty, StatisticsError will be raised.
    """
    if iter(data) is data:
        data = list(data)
    n = len(data)
    if n < 1:
        raise StatisticsError('mean requires at least one data point')
    T, total, count = _sum(data)
    assert count == n
    return _convert(total / n, T)
```

### 2.7.2 迭代器对象和可迭代对象的区别

在 Python 中，**迭代器对象** 和 **可迭代对象** 是两个不同的概念。

##### 1. 可迭代对象（Iterable）
- **定义**：可迭代对象是实现了 `__iter__()` 方法的对象。这个方法返回一个迭代器对象。
- **特点**：
  - 可以通过 `for` 循环遍历。
  - 可以传递给内置函数 `iter()` 来获取迭代器对象。

- **常见类型**：
  - 列表 (`list`)
  - 元组 (`tuple`)
  - 字符串 (`str`)
  - 字典 (`dict`)
  - 集合 (`set`)
  - 文件对象
  - 自定义的可迭代类

**示例**：
```python
# 列表是一个可迭代对象
my_list = [1, 2, 3]
for item in my_list:
    print(item)

# 使用 iter() 获取迭代器对象
iterator = iter(my_list)  # 大多数情况下，iter(my_list) 等价于 my_list.__iter__()
print(next(iterator))  # 输出 1
print(next(iterator))  # 输出 2
```

##### 2. 迭代器对象（Iterator）
- **定义**：迭代器对象是实现了 `__next__()` 方法的对象。每次调用 `next()` 函数时，它会返回下一个值，直到没有更多元素为止，此时会抛出 `StopIteration` 异常。

- **特点**：
  - 必须实现 `__iter__()` 方法，返回自身（即 `self`）。
  - 每次调用 `next()` 返回序列中的下一个值。
  - 一旦遍历结束，再次调用 `next()` 会抛出 `StopIteration` 异常。
- **常见类型**：
  - 由 `iter()` 函数从可迭代对象生成的对象。
  - 生成器（Generator）

**示例**：
```python
# 创建一个迭代器对象
my_iterator = iter([1, 2, 3])

# 使用 next() 获取下一个值
print(next(my_iterator))  # 输出 1
print(next(my_iterator))  # 输出 2
print(next(my_iterator))  # 输出 3
# print(next(my_iterator))  # 抛出 StopIteration 异常
```

##### 3. 区别与联系
- **可迭代对象** 是可以被迭代的对象，但不一定能直接调用 `next()` 方法。
- **迭代器对象** 是专门用于迭代的对象，可以直接调用 `next()` 方法。
- **关系**：每个可迭代对象都可以通过 `iter()` 函数转换为迭代器对象。迭代器对象本身也是一个可迭代对象，因为它实现了 `__iter__()` 方法并返回自身。

##### 4. 特殊情况：`if iter(data) is data`
- **解释**：这行代码检查 `data` 是否既是可迭代对象又是迭代器对象，并且 `iter(data)` 返回的就是 `data` 本身。
- **应用场景**：通常用于判断 `data` 是否已经是迭代器对象，而不是普通的可迭代对象。例如，在某些情况下，你可能需要传入可迭代对象作为参数，避免重复创建迭代器对象。

# 3.Extras & Tensorboard logging

## 3.1 extras 的定义

```python
class DirectRLEnv(gym.Env):
    # ...
    def __init__(self, cfg: DirectRLEnvCfg, render_mode: str | None = None, **kwargs):
        # ...
        # allocate dictionary to store metrics
        self.extras = {}
        # ...
    def reset(self, seed: int | None = None, options: dict[str, Any] | None = None) -> tuple[VecEnvObs, dict]:
        # ...
        # reset state of scene
        indices = torch.arange(self.num_envs, dtype=torch.int64, device=self.device)
        self._reset_idx(indices)  # 传入indices有点奇怪，不知道哪里会使用这个reset函数，后面发现被RslRlVecEnvWrapper()重写了
        # ...
        # return observations
        return self._get_observations(), self.extras
    def step(self, action: torch.Tensor) -> VecEnvStepReturn:
        """Returns:
            A tuple containing the observations, rewards, resets (terminated and truncated) and extras.
        """
        # ...
        # return observations, rewards, resets and extras
        return self.obs_buf, self.reward_buf, self.reset_terminated, self.reset_time_outs, self.extras
    # ...
    def _reset_idx(self, env_ids: Sequence[int]):
        """Reset environments based on specified indices.
        """
        self.scene.reset(env_ids)
        # ...
    # ...
```

## 3.2 为extras添加自定义的信息
```python
self.extras["log"] = dict()
self.extras["log"].update(your_log_dict)
```

.e.g
```python
class AnymalCEnv(DirectRLEnv):
    cfg: AnymalCFlatEnvCfg | AnymalCRoughEnvCfg

    def __init__(self, cfg: AnymalCFlatEnvCfg | AnymalCRoughEnvCfg, render_mode: str | None = None, **kwargs):
        super().__init__(cfg, render_mode, **kwargs)
        # ...
        # Logging
        self._episode_sums = {
            key: torch.zeros(self.num_envs, dtype=torch.float, device=self.device)
            for key in [
                "track_lin_vel_xy_exp",
                "track_ang_vel_z_exp",
                "lin_vel_z_l2",
                "ang_vel_xy_l2",
                "dof_torques_l2",
                "dof_acc_l2",
                "action_rate_l2",
                "feet_air_time",
                "undesired_contacts",
                "flat_orientation_l2",
            ]
        }
        # ...
    # ...
    def _get_rewards(self) -> torch.Tensor:
        # ...
        rewards = {
            "track_lin_vel_xy_exp": lin_vel_error_mapped * self.cfg.lin_vel_reward_scale * self.step_dt,
            "track_ang_vel_z_exp": yaw_rate_error_mapped * self.cfg.yaw_rate_reward_scale * self.step_dt,
            "lin_vel_z_l2": z_vel_error * self.cfg.z_vel_reward_scale * self.step_dt,
            "ang_vel_xy_l2": ang_vel_error * self.cfg.ang_vel_reward_scale * self.step_dt,
            "dof_torques_l2": joint_torques * self.cfg.joint_torque_reward_scale * self.step_dt,
            "dof_acc_l2": joint_accel * self.cfg.joint_accel_reward_scale * self.step_dt,
            "action_rate_l2": action_rate * self.cfg.action_rate_reward_scale * self.step_dt,
            "feet_air_time": air_time * self.cfg.feet_air_time_reward_scale * self.step_dt,
            "undesired_contacts": contacts * self.cfg.undersired_contact_reward_scale * self.step_dt,
            "flat_orientation_l2": flat_orientation * self.cfg.flat_orientation_reward_scale * self.step_dt,
        }
        reward = torch.sum(torch.stack(list(rewards.values())), dim=0)
        # Logging
        for key, value in rewards.items():
            self._episode_sums[key] += value
        return reward
    # ...
    def _reset_idx(self, env_ids: torch.Tensor | None):
        # ...
        # Logging
        extras = dict()
        for key in self._episode_sums.keys():
            episodic_sum_avg = torch.mean(self._episode_sums[key][env_ids])
            extras["Episode_Reward/" + key] = episodic_sum_avg / self.max_episode_length_s
            self._episode_sums[key][env_ids] = 0.0
        self.extras["log"] = dict()
        self.extras["log"].update(extras)
        extras = dict()
        extras["Episode_Termination/base_contact"] = torch.count_nonzero(self.reset_terminated[env_ids]).item()
        extras["Episode_Termination/time_out"] = torch.count_nonzero(self.reset_time_outs[env_ids]).item()
        self.extras["log"].update(extras)
```

## 3.3 extras 的包装和改造
`extras["log"]` 和 `extras["observations"]`、`extras["time_outs"]`

```python
# ~/IsaacLab/source/extensions/omni.isaac.lab_tasks/omni/isaac/lab_tasks/utils/wrappers/rsl_rl/vecenv_wrapper.py
class RslRlVecEnvWrapper(VecEnv):
    # ...
    def reset(self) -> tuple[torch.Tensor, dict]:  # noqa: D102  # 忽略缺少公共方法文档字符串的警告
        # reset the environment
        obs_dict, _ = self.env.reset()  # 前面定义的extra被丢弃了
        # return observations
        return obs_dict["policy"], {"observations": obs_dict}
    
    def step(self, actions: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, dict]:
        # record step information
        obs_dict, rew, terminated, truncated, extras = self.env.step(actions)
        # compute dones for compatibility with RSL-RL
        dones = (terminated | truncated).to(dtype=torch.long)
        # move extra observations to the extras dict
        obs = obs_dict["policy"]
        extras["observations"] = obs_dict
        # move time out information to the extras dict
        # this is only needed for infinite horizon tasks
        if not self.unwrapped.cfg.is_finite_horizon:
            extras["time_outs"] = truncated
        # return the step information
        return obs, rew, dones, extras
    #...
```

## 3.4 in OnPolicyRunner
`ep_infos = []`，每一个step都有 `ep_infos.append(infos["log"])`

但是一轮iteration（16steps .e.g）才会`self.log(locals())`，随后`ep_infos.clear()`

### 3.4.1 log()

```python
class OnPolicyRunner:
    #...
    def log(self, locs: dict, width: int = 80, pad: int = 35):
        self.tot_timesteps += self.num_steps_per_env * self.env.num_envs
        self.tot_time += locs["collection_time"] + locs["learn_time"]
        iteration_time = locs["collection_time"] + locs["learn_time"]

        ep_string = ""
        if locs["ep_infos"]:
            for key in locs["ep_infos"][0]:
                infotensor = torch.tensor([], device=self.device)  # 对于每个键，初始化一个空的张量infotensor，用于存储该键对应的值
                for ep_info in locs["ep_infos"]:  # 遍历每个step，len = num_steps_per_env (16 .e.g)
                    # handle scalar and zero dimensional tensor infos
                    if key not in ep_info:
                        continue  # 如果当前step信息中不存在该键，则跳过
                    if not isinstance(ep_info[key], torch.Tensor):
                        ep_info[key] = torch.Tensor([ep_info[key]])  # 如果该键对应的值不是张量，则将其转换为张量
                    if len(ep_info[key].shape) == 0:
                        ep_info[key] = ep_info[key].unsqueeze(0)  # 如果该键对应的是零维张量（标量），则升为一维张量，确保后续的cat能够正常工作
                    infotensor = torch.cat((infotensor, ep_info[key].to(self.device)))  # 将该键每个step对应的值与infotensor进行拼接
                value = torch.mean(infotensor)  # 计算该键本轮所有step的平均值，所以一些键本应是整数的值，成了小数
                # log to logger and terminal
                if "/" in key:
                    self.writer.add_scalar(key, value, locs["it"])
                    ep_string += f"""{f'{key}:':>{pad}} {value:.4f}\n"""
                else:
                    self.writer.add_scalar("Episode/" + key, value, locs["it"])
                    ep_string += f"""{f'Mean episode {key}:':>{pad}} {value:.4f}\n"""
        # ...
        # 一些默认统计信息的计算和输出
```

In [6]:
ep_infos = []
if ep_infos:
    print("ep_infos = true")
else:
    print(ep_infos)

[]


#### 零维张量（0-dimensional tensor）

PyTorch中零维张量也称为标量张量Scalar tensor。零维张量没有形状（shape），通常用于表示单个数值。

**零维张量的特点：**
- **形状**：`torch.Size([])`，表示没有维度。
- **大小**：包含一个单一的值。
- **操作**：可以像普通标量一样进行数学运算，并且可以通过 `.item()` 方法获取其内部的 Python 标量值。


In [10]:
import torch
# 创建一个零维张量
scalar_tensor = torch.tensor(3.14)
print(f"Scalar tensor: {scalar_tensor}")
print(f"Shape: {scalar_tensor.shape}")  # 输出: Shape: torch.Size([])
print(f"Value: {scalar_tensor.item()}")  # 输出: Value: 3.14

Scalar tensor: 3.140000104904175
Shape: torch.Size([])
Value: 3.140000104904175


在上述代码中
```python
if len(ep_info[key].shape) == 0:
    ep_info[key] = ep_info[key].unsqueeze(0)
```
检查 `ep_info[key]` 是否为零维张量，如果是，则使用 `unsqueeze(0)` 方法为其增加一个维度，使其变为一维张量。这一步骤是为了确保后续的拼接操作（如 `torch.cat`）能够正常工作，因为 `torch.cat` 需要所有输入张量具有相同的维度。

#### 嵌套的格式化字符串
`ep_string += f"""{f'{key}:':>{pad}} {value:.4f}\n"""`

- 内层的 `f'...':>{pad}`，用于将变量插入到字符串中
- 外层的 `f"""..."""`，用于进一步格式化字符串
  - `:>{pad}` 右对齐并填充空白字符，使整个字符串的宽度达到pad指定的长度，默认右对齐，即`:{pad}`
  - 左对齐填充*，则`:*<{pad}`
  - `{value:.4f}` 将 value 变量格式化为浮点数，保留四位小数

### 3.4.2
默认的log信息

- loss
  - Loss/value_function `mean_value_loss, mean_surrogate_loss = self.alg.update()`
  - Loss/surrogate `mean_value_loss, mean_surrogate_loss = self.alg.update()`
  - Loss/learning_rate `self.alg.learning_rate`
- Policy
  - Policy/mean_noise_std `self.alg.actor_critic.std.mean().item()`
- Perf
  - Perf/total_fps `int(self.num_steps_per_env * self.env.num_envs / (collection_time + learn_time))`
  - Perf/collection time `collection_time`
  - Perf/learning_time `learn_time`
- Train
  - Train/mean_reward `statistics.mean(rewbuffer)`
  - Train/mean_episode_length `statistics.mean(lenbuffer)`
  - Train/mean_reward/time `statistics.mean(rewbuffer)`  横坐标不再是轮次`it`，而是时间`tot_time`
  - Train/mean_episode_length/time `statistics.mean(lenbuffer)`  横坐标不再是轮次`it`，而是时间`tot_time`


Terminal log_string += ep_string

log_sting += ↓

---

Terminal额外的信息：
- Total timesteps: `self.tot_timesteps += self.num_steps_per_env * self.env.num_envs`
- Iteration time: `iteration_time = collection_time + learn_time`
- Total time: `self.tot_time += collection_time + learn_time`
- ETA: `self.tot_time / (it + 1) * (num_learning_iterations - it)` 估计剩余时间

# 4. PPO

```python
class OnPolicyRunner:
    """On-policy runner for training and evaluation."""

    def __init__(self, env: VecEnv, train_cfg, log_dir=None, device="cpu"):
        self.cfg = train_cfg  # agent_cfg.to_dict() `runner = OnPolicyRunner(env, agent_cfg.to_dict(), log_dir=log_dir, device=agent_cfg.device)`
        self.alg_cfg = train_cfg["algorithm"]
        self.policy_cfg = train_cfg["policy"]
        self.device = device
        self.env = env
        obs, extras = self.env.get_observations()
        num_obs = obs.shape[1]
        if "critic" in extras["observations"]:
            num_critic_obs = extras["observations"]["critic"].shape[1]
        else:
            num_critic_obs = num_obs
        actor_critic_class = eval(self.policy_cfg.pop("class_name"))  # ActorCritic
        actor_critic: ActorCritic | ActorCriticRecurrent = actor_critic_class(
            num_obs, num_critic_obs, self.env.num_actions, **self.policy_cfg
        ).to(self.device)
        alg_class = eval(self.alg_cfg.pop("class_name"))  # PPO
        self.alg: PPO = alg_class(actor_critic, device=self.device, **self.alg_cfg)
        self.num_steps_per_env = self.cfg["num_steps_per_env"]
        self.save_interval = self.cfg["save_interval"]
        self.empirical_normalization = self.cfg["empirical_normalization"]
```